Импортируем нужные библиотеки

In [1]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers import MLFlowLogger

Импортируем нужные классы

In [2]:
import config
from dataset import DataModule
from model import RankingModel

Задаем сиды, чтобы можно было воспроизводить результаты

In [3]:
torch.random.manual_seed(42)
pl.seed_everything(42)

Seed set to 42


42

In [4]:
logger = TensorBoardLogger(save_dir="tb_logs", name="ranking_model")

In [5]:
mlf_logger = MLFlowLogger(experiment_name=config.EXPERIMENT_NAME, tracking_uri=config.TRACKING_URL, log_model=config.LOG_MODEL)

Создаем необходимые инстанты: данные, модель, учитель.

In [6]:
dm = DataModule(config.TRAIN_DATA_PATH, config.TEST_DATA_PATH, config.BATCH_SIZE, config.NUM_WORKERS, config.TRAIN_VAL_RATIO)
model = RankingModel(config.INPUT_SIZE, config.LEARNING_RATE)
trainer = pl.Trainer(
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=config.MIN_EPOCHS,
        max_epochs=config.MAX_EPOCHS,
        logger=[logger, mlf_logger],
    )

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Обучаем и тестируем модель.

In [7]:
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)


  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 960   
1 | fc2  | Linear | 2.1 K 
2 | fc3  | Linear | 33    
--------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 189/189 [00:01<00:00, 132.62it/s, v_num=765b]    

`Trainer.fit` stopped: `max_epochs=10` reached.


Validation DataLoader 0: 100%|██████████| 48/48 [00:00<00:00, 223.73it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            0.08952904492616653
        val_ndcg            0.4947589039802551
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 24/24 [00:00<00:00, 185.34it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.10005228221416473
        test_ndcg  

[{'test_loss': 0.10005228221416473, 'test_ndcg': 0.5590013861656189}]

Сохраняем модель

In [8]:
script = model.to_torchscript()
torch.jit.save(script, config.MODEL_PATH)